In [21]:
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect
from sqlalchemy import Integer, Column, Float, String
import numpy as np

In [22]:
import pandas as pd
import os
import csv
import requests
#from config import api_key
from pprint import pprint
import matplotlib.pyplot as plt


In [23]:
input_file= os.path.join("TMDB_OMDB-3.csv")
df_new=pd.read_csv('TMDB_OMDB-3.csv', encoding = 'utf8')
df_new.head()


,show_id,type,title,director,cast,country,date_added,Netflix release year,rating,duration,listed_in,description,IMDB rating,Rotten Tomatoes rating,Award,Released Date,Budget,Movie ID,Revenue
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,3,36,NaN,2-Aug-19,0,601131.0,1442504
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,5,0,NaN,NaN,0,415722.0,0
2,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,8-Sep-17,2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...,5,31,2 nominations.,14-Aug-13,0,164337.0,0
3,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",6,29,6 nominations.,17-Oct-14,7000000,262543.0,0
4,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden",8-Sep-17,2014,R,90 min,"Action & Adventure, Thrillers",A struggling couple can't believe their luck w...,5,12,NaN,21-Aug-15,0,262338.0,0


In [95]:
engine = create_engine("postgres://pgadmin@pg-srv-001:ucN-xZRL3NsaBjvG2tcw1gPcsNeS5Xfw@pg-srv-001.postgres.database.azure.com:5432/netflix",connect_args={'sslmode':'require'})

conn=engine.connect()
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['listed_in', 'title', 'players', 'pg_rating', 'country', 'player_title']

In [96]:
inspector = inspect(engine)
inspector.get_table_names()

['listed_in',
 'director',
 'title',
 'players',
 'pg_rating',
 'country',
 'player_title']

In [26]:
title_df = pd.DataFrame(df_new,columns=['show_id','show_type','title','date_added','Netflix release year','duration','description','IMDB rating','Rotten Tomatoes rating','Award','Released Date','Budget','Movie ID','Revenue'])

title_df = title_df.reset_index(drop=True)
title_df.head()

,show_id,show_type,title,date_added,Netflix release year,duration,description,IMDB rating,Rotten Tomatoes rating,Award,Released Date,Budget,Movie ID,Revenue
0,81145628,NaN,Norm of the North: King Sized Adventure,9-Sep-19,2019,90 min,Before planning an awesome wedding for his gra...,3,36,NaN,2-Aug-19,0,601131.0,1442504
1,80117401,NaN,Jandino: Whatever it Takes,9-Sep-16,2016,94 min,Jandino Asporaat riffs on the challenges of ra...,5,0,NaN,NaN,0,415722.0,0
2,80163890,NaN,Apaches,8-Sep-17,2016,1 Season,A young journalist is forced into a life of cr...,5,31,2 nominations.,14-Aug-13,0,164337.0,0
3,70304989,NaN,Automata,8-Sep-17,2014,110 min,"In a dystopian future, an insurance adjuster f...",6,29,6 nominations.,17-Oct-14,7000000,262543.0,0
4,70304990,NaN,Good People,8-Sep-17,2014,90 min,A struggling couple can't believe their luck w...,5,12,NaN,21-Aug-15,0,262338.0,0


In [7]:
title_df["description"] = title_df["description"].str.replace("'","''")
title_df["title"] = title_df["title"].str.replace("'","''")

In [9]:
show_id_list = list(title_df['show_id'])
show_type_list = list(title_df['show_type'])
title_list = list(title_df['title'])
date_added_list = list(title_df['date_added'])
Netflix_release_year_list = list(title_df['Netflix release year'])
duration_list = list(title_df['duration'])
description_list = list(title_df['description'])
IMDB_rating_list = list(title_df['IMDB rating'])
Rotten_Tomatoes_rating_list = list(title_df['Rotten Tomatoes rating'])
Award_list = list(title_df['Award'])
Released_Date_list = list(title_df['Released Date'])
Budget_list = list(title_df['Budget'])
Movie_ID_list = list(title_df['Movie ID'])
Revenue_list = list(title_df['Revenue'])


for i in range (len (show_id_list)):
    statement = f"INSERT INTO title VALUES ({show_id_list[i]},'{show_type_list[i]}','{title_list[i]}','{date_added_list[i]}','{Netflix_release_year_list[i]}','{duration_list[i]}','{description_list[i]}','{IMDB_rating_list[i]}',{Rotten_Tomatoes_rating_list[i]},'{Award_list[i]}','{Released_Date_list[i]}',{Budget_list[i]},{Revenue_list[i]},'{Movie_ID_list[i]}')"
    #print (statement)
    conn.execute(statement)
    #      print (show_id_list,show_type_list,title_list,date_added_list,Netflix_release_year_list,duration_list,description_list,IMDB_rating_list,Rotten_Tomatoes_rating_list,Award_list,Released_Date_list,Movie_ID_list,Revenue_list)


In [10]:
Director_df = pd.DataFrame(df_new,columns=['director'])
#Director_df = Director_df.set_index("director")
Director_df = Director_df.reset_index(drop=True)
Director_df.head()

,director
0,"Richard Finn, Tim Maltby"
1,NaN
2,NaN
3,Gabe Ibáñez
4,Henrik Ruben Genz


In [11]:
Director_df.director = Director_df.director.str.split(',')
Director_df = Director_df.explode('director').reset_index(drop=True)
Director_df.drop_duplicates(subset=['director'], inplace=True)
Director_df.dropna(subset=['director'],inplace=True)
#Director_df = Director_df[Director_df.director != "None"]
Director_df

,director
0,Richard Finn
1,Tim Maltby
4,Gabe Ibáñez
5,Henrik Ruben Genz
6,José Miguel Contreras
...,...
3734,Kike Maíllo
3735,G.J. Echternkamp
3736,Zatella Beatty
3766,Glen Winter


In [12]:
Director_df["director"] = Director_df["director"].str.replace("'","''")

In [27]:
for cols in Director_df['director']: 
    statement = f"INSERT INTO Director (director_name) VALUES ('{cols}')"
    conn.execute(statement)

In [28]:
Listed_in_df = pd.DataFrame(df_new,columns=['listed_in'])
Listed_in_df = Listed_in_df.reset_index(drop=True)
Listed_in_df.head()

,listed_in
0,"Children & Family Movies, Comedies"
1,Stand-Up Comedy
2,"Crime TV Shows, International TV Shows, Spanis..."
3,"International Movies, Sci-Fi & Fantasy, Thrillers"
4,"Action & Adventure, Thrillers"


In [29]:
Listed_in_df.listed_in = Listed_in_df.listed_in.str.split(',')
Listed_in_df = Listed_in_df.explode('listed_in').reset_index(drop=True)
Listed_in_df.drop_duplicates(subset=['listed_in'], inplace=True)
Listed_in_df.dropna(subset=['listed_in'],inplace=True)
Listed_in_df

,listed_in
0,Children & Family Movies
1,Comedies
2,Stand-Up Comedy
3,Crime TV Shows
4,International TV Shows
...,...
3180,Romantic Movies
5223,Kids' TV
7115,TV Sci-Fi & Fantasy
7134,Classic & Cult TV


In [32]:

Listed_in_df['listed_in'] = Listed_in_df['listed_in'].str.strip()
Listed_in_df.drop_duplicates (subset = ['listed_in'], inplace = True) 
Listed_in_df.reset_index(inplace = True )
Listed_in_df

,level_0,index,listed_in
0,0,0,Children & Family Movies
1,1,1,Comedies
2,2,2,Stand-Up Comedy
3,3,3,Crime TV Shows
4,4,4,International TV Shows
5,5,5,Spanish-Language TV Shows
6,6,6,International Movies
7,7,7,Sci-Fi & Fantasy
8,8,8,Thrillers
9,9,9,Action & Adventure


In [38]:
Listed_in_df["listed_in"] = Listed_in_df["listed_in"].str.replace("'","''")

In [39]:
i =1
for cols in Listed_in_df['listed_in']: 
    statement = f"INSERT INTO listed_in VALUES ({i},'{cols}')"
    conn.execute(statement)
    i=i+1

In [52]:
cast_df = pd.DataFrame(df_new,columns=['cast'])
cast_df = cast_df.reset_index(drop=True)
cast_df.head()

,cast
0,"Alan Marriott, Andrew Toth, Brian Dobson, Cole..."
1,Jandino Asporaat
2,"Alberto Ammann, Eloy Azorín, Verónica Echegui,..."
3,"Antonio Banderas, Dylan McDermott, Melanie Gri..."
4,"James Franco, Kate Hudson, Tom Wilkinson, Omar..."


In [53]:
cast_df.cast =cast_df.cast.str.split(',')
cast_df = cast_df.explode('cast').reset_index(drop=True)
cast_df.drop_duplicates(subset=['cast'], inplace=True)
cast_df.dropna(subset=['cast'],inplace=True)
cast_df

,cast
0,Alan Marriott
1,Andrew Toth
2,Brian Dobson
3,Cole Howard
4,Jennifer Cameron
...,...
24506,Craig Behenna
24507,Charlotte Hamlyn
24508,Stavroula Mountzouris
24509,Aletheia Burney


In [54]:
cast_df["cast"] = cast_df["cast"].str.replace("'","''")

In [55]:
cast_df["cast"] = cast_df["cast"].str.strip()
cast_df.drop_duplicates (subset = ['cast'], inplace = True) 
cast_df.reset_index(inplace = True )
cast_df

,index,cast
0,0,Alan Marriott
1,1,Andrew Toth
2,2,Brian Dobson
3,3,Cole Howard
4,4,Jennifer Cameron
...,...,...
16290,24506,Craig Behenna
16291,24507,Charlotte Hamlyn
16292,24508,Stavroula Mountzouris
16293,24509,Aletheia Burney


In [56]:
i =1
for cols in cast_df['cast']: 
    statement = f"INSERT INTO players VALUES ({i},'{cols}')"
    conn.execute(statement)
    i=i+1


In [57]:
pg_rating_df = pd.DataFrame(df_new,columns=['rating'])
pg_rating_df = pg_rating_df.reset_index(drop=True)
pg_rating_df.head()

,rating
0,TV-PG
1,TV-MA
2,TV-MA
3,R
4,R


In [58]:
pg_rating_df.drop_duplicates(subset=['rating'], inplace=True)
pg_rating_df.dropna(subset=['rating'],inplace=True)
pg_rating_df

,rating
0,TV-PG
1,TV-MA
3,R
7,TV-14
10,PG-13
12,NR
28,PG
42,TV-Y7
43,TV-G
86,TV-Y


In [59]:
for cols in pg_rating_df['rating']:
    statement = f"INSERT INTO pg_rating (pg_rating_name) VALUES ('{cols}')"
    conn.execute(statement)

In [62]:
Country_df = pd.DataFrame(df_new,columns=['country'])
Country_df = Country_df.reset_index(drop=True)
Country_df.head(50)

,country
0,"United States, India, South Korea, China"
1,United Kingdom
2,Spain
3,"Bulgaria, United States, Spain, Canada"
4,"United States, United Kingdom, Denmark, Sweden"
5,NaN
6,"Netherlands, Belgium, United Kingdom, United S..."
7,United States
8,"France, Belgium"
9,"United States, Uruguay"


In [67]:
input_file= os.path.join("countries.csv")
Country_df=pd.read_csv('countries.csv', encoding = 'utf8')
Country_df.tail()


,country,latitude,longitude,name
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia
244,ZW,-19.015438,29.154857,Zimbabwe


In [75]:
Country_df["name"] = Country_df["name"].str.replace("'","''")

In [79]:
fill_values = {'country':"N/A"}
Country_df.fillna(value=fill_values, inplace=True)

print(Country_df.loc[Country_df['name']== 'Namibia'])



    country  latitude  longitude     name
156     N/A -22.95764   18.49041  Namibia


In [80]:
country_list = list(Country_df['country'])
lat_list = list(Country_df['latitude'])
long_list = list(Country_df['longitude'])
name_list= list(Country_df['name'])


for i in range (len (country_list)):

    statement = f"INSERT INTO country VALUES  ({i},'{name_list[i]}','{country_list[i]}','{round(long_list[i],6)}','{round(lat_list[i],6)}')"
    conn.execute(statement)

In [ ]:
netflix_titles_Cast = pd.DataFrame ({"Titles": df_new["show_id"],
                                          "Cast": df_new["cast"]}) 
netflix_titles_Cast.Cast = netflix_titles_Cast.Cast.str.split(', ')
netflix_titles_Cast = netflix_titles_Cast.explode('Cast').reset_index(drop=True)
netflix_titles_Cast.dropna(subset=['Cast'], inplace=True)

In [110]:
netflix_titles_Cast["Cast"] = netflix_titles_Cast["Cast"].str.replace("'","''")

,Titles,Cast
0,81145628,Alan Marriott
1,81145628,Andrew Toth
2,81145628,Brian Dobson
3,81145628,Cole Howard
4,81145628,Jennifer Cameron
...,...,...
24511,70153404,Courteney Cox
24512,70153404,Lisa Kudrow
24513,70153404,Matt LeBlanc
24514,70153404,Matthew Perry


In [113]:
netflix_titles_Cast.drop_duplicates (subset = ['Titles','Cast'], inplace = True) 
netflix_titles_Cast.head()

,Titles,Cast
0,81145628,Alan Marriott
1,81145628,Andrew Toth
2,81145628,Brian Dobson
3,81145628,Cole Howard
4,81145628,Jennifer Cameron


In [117]:
player_name_list = list(netflix_titles_Cast['Cast'])
show_id_list = list(netflix_titles_Cast['Titles'])

for i in range(len(player_name_list)):
    statement_player_id = engine.execute(f"select player_id from players where player_name = '{player_name_list[i]}'").fetchall()
    statement = f"INSERT INTO player_title (player_id, show_id) values ({statement_player_id[0][0]},{show_id_list[i]})"
    conn.execute(statement)

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: INSERT INTO player_title (player_id, show_id) values (8893,80157207)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
Def 
for col1, col2 in title_df.iterrows():
    print(col2)
    #statement = f"INSERT INTO title (show_id,show_type,title,date_added,Netflix release year,duration,description,IMDB rating,Rotten Tomatoes rating,Award,Released Date,Movie ID,Revenue) VALUES ('{cols}')"
    #conn.execute(statement)

In [ ]:
columns = inspector.get_columns('title')
columns

In [ ]:
for cols in columns: 
    print(cols['name'], cols['type'])

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Title = Base.classes.title

In [ ]:
session = Session(engine)

In [ ]:
results = session.query(Title.show_id, Title.award, Title.movie_id).order_by(Title.movie_id.desc()).all()

In [ ]:
print(results)

In [ ]:
results_2 = session.query(Title.director).all()
print(results_2)
#Director_df = pd.DataFrame(results_2,columns=['director'])
#prcp_df = prcp_df.set_index("director")
#Director_df.head()

In [ ]:
Director_df = pd.DataFrame(results_2,columns=['director'])
#Director_df = Director_df.set_index("director")
Director_df = Director_df.reset_index(drop=True)
Director_df.head()

In [ ]:
from sqlalchemy import insert

In [ ]:
for cols in results_2: 
    print(cols[0])

In [ ]:
engine.execute("Select * from Director").fetchall()

In [ ]:
Director_df.director = Director_df.director.str.split(',')
Director_df = Director_df.explode('director').reset_index(drop=True)
Director_df.drop_duplicates(subset=['director'], inplace=True)
Director_df.dropna(subset=['director'],inplace=True)
#Director_df = Director_df[Director_df.director != "None"]
Director_df

In [ ]:
Director_df["director"] = Director_df["director"].str.replace("'","''")

In [ ]:
for cols in Director_df['director']: 
    statement = f"INSERT INTO Director (director_name) VALUES ('{cols}')"
    conn.execute(statement)

In [ ]:
#Country
results_3 = session.query(Title.country).all()
print(results_3)

In [ ]:
#Country
Country_df = pd.DataFrame(results_3,columns=['country'])
Country_df = Country_df.reset_index(drop=True)
Country_df.head()

In [ ]:
Country_df.country = Country_df.country.str.split(',')
Country_df = Country_df.explode('country').reset_index(drop=True)
Country_df.drop_duplicates(subset=['country'], inplace=True)
Country_df.dropna(subset=['country'],inplace=True)
#Director_df = Director_df[Director_df.director != "None"]
Country_df

In [ ]:
for h in Country_df['country']: 
    statement_2 = f"INSERT INTO country (country_name) VALUES ('{h}')"
    conn.execute(statement_2)
    #print(col)

In [ ]:
#Rating
results_4 = session.query(Title.rating).all()
print(results_4)

In [ ]:
pg_rating_df = pd.DataFrame(results_4,columns=['rating'])
pg_rating_df = pg_rating_df.reset_index(drop=True)
pg_rating_df.head()

In [ ]:
for cols in results_4:
    print(cols[0])

In [ ]:
engine.execute("Select * from pg_rating").fetchall()

In [ ]:
pg_rating_df.drop_duplicates(subset=['rating'], inplace=True)
pg_rating_df.dropna(subset=['rating'],inplace=True)
pg_rating_df

In [ ]:
for cols in pg_rating_df['rating']:
    statement = f"INSERT INTO pg_rating (pg_rating_name) VALUES ('{cols}')"
    conn.execute(statement)

In [ ]:
######### New Code #######
netflix_titles_path = "TMDB_OMDB.csv"

# Read the mouse data and the study results
netflix_titles = pd.read_csv(netflix_titles_path)
netflix_titles.replace('', np.nan, inplace=True) 
netflix_titles.drop_duplicates(subset=['show_id'], inplace=True)
#netflix_titles.dropna(subset=['Movie ID','director', 'cast', 'country', 'listed_in'], inplace=True)
#netflix_titles = netflix_titles_temp.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

netflix_titles

In [ ]:
netflix_titles_countries = pd.DataFrame ({"Titles": netflix_titles["show_id"],
                                          "Country": netflix_titles["country"]}) 
netflix_titles_countries.Country = netflix_titles_countries.Country.str.split(', ')
netflix_titles_countries = netflix_titles_countries.explode('Country').reset_index(drop=True)
netflix_titles_countries.dropna(subset=['Country'], inplace=True)
netflix_titles_countries.head(30)

In [ ]:
netflix_titles_listed_in = pd.DataFrame ({"Titles": netflix_titles["show_id"],
                                          "Listed_In": netflix_titles["listed_in"]}) 
netflix_titles_listed_in.Listed_In = netflix_titles_listed_in.Listed_In.str.split(', ')
netflix_titles_listed_in = netflix_titles_listed_in.explode('Listed_In').reset_index(drop=True)
netflix_titles_listed_in.dropna(subset=['Listed_In'], inplace=True)
netflix_titles_listed_in.head(30)

In [ ]:
netflix_titles_directors = pd.DataFrame ({"Titles": netflix_titles["show_id"],
                                          "directors": netflix_titles["director"]}) 
netflix_titles_directors.directors = netflix_titles_directors.directors.str.split(', ')
netflix_titles_directors = netflix_titles_directors.explode('directors').reset_index(drop=True)
netflix_titles_directors.dropna(subset=['directors'], inplace=True)
netflix_titles_directors.head(30)

In [ ]:
netflix_titles_Cast = pd.DataFrame ({"Titles": netflix_titles["show_id"],
                                          "Cast": netflix_titles["cast"]}) 
netflix_titles_Cast.Cast = netflix_titles_Cast.Cast.str.split(', ')
netflix_titles_Cast = netflix_titles_Cast.explode('Cast').reset_index(drop=True)
netflix_titles_Cast.dropna(subset=['Cast'], inplace=True)
netflix_titles_Cast.head(30)



